In [8]:
import pandas as pd
import numpy as np

from utils.clean_utils import clean_dataframe

# Read Data

In [2]:
df_raw = pd.read_csv('data/feats_out.csv')

/Users/seramirchandani/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,24,29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Clean Data

In [9]:
features, labels, identifiers = clean_dataframe(df_raw)

In [13]:
len(features.columns)

1663